In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.
import csv
import pandas as pd

file = '../input/monthly_salary_brazil.csv'
f = open(file,'rt')
reader = csv.reader(f)

#once contents are available, I then put them in a list
csv_list = []
for l in reader:
    csv_list.append(l)
f.close()
#now pandas has no problem getting into a df
df = pd.DataFrame(csv_list)
#Solving the problem on line 845 and 847
i=1
while (i<10):
    df.loc[845,i] = df.loc[845,i+1]
    df.loc[847,i] = df.loc[847,i+1]
    i += 1
df.drop(labels=10,axis=1,inplace=True)

df.columns = df.iloc[0]
df = df.reindex(df.index.drop(0),)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats
sns.set_style("darkgrid")
%matplotlib inline

# Functions

In [ ]:
def dist_info(data,target):
    print("="*90)
    print('Distribution info:')
    print("Mean: %f" %data[target].mean())
    print("STD: %f" %data[target].std())
    print("Skewness:%f" %data[target].skew())
    print("Kurtosis: %f" %data[target].kurt())
    print("="*90)

In [ ]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent = (100*(data.isnull().sum()/data.isnull().count())).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

In [ ]:
def Delta_mapping(p):
    mapping = [(15,'Under 15%'),(30,'15-30%'),(60,'30-60%'),(90,'60-90%'),
               (120,'90-120%'),(180,'120-180%'),(240,'180-240%'),(400,'240-400%'),
               (800,'400-800%')]
    for check, value in mapping:
        if p <= check:
            return value

# Knowing the Data

$\textbf{Data Dictionary}$ 

* $\textbf{Id}$: Person identification

* $\textbf{Job}$: Job description.

* $\textbf{Sector}$: Public sector.

* $\textbf{Month_Salary}$: Regular monthly salary.

* $\textbf{13_Salary}$:The thirteenth salary is paid to employees according to different terms of law or contract of employment. Mostly, it is equivalent to one full monthly salary and paid in December (the 12th month) around Christmas and is traditionally called a Christmas bonus.

* $\textbf{indemnity}$: eventual variable.

* $\textbf{extra_salary}$: eventual variable.

* $\textbf{discount_salary}$: eventual variable.

* $\textbf{total_salary}$: Actual salary received after a month of work. 

 **Additional Variable**
 
*  **label_job** : Job label identification
* **Delta_S** :  Simulate the percentage of income tax for each person ID
* **Over_limit** : Dummy variable to identify if the public worker received a month salary over the limit.

# Problem Motivation

I will just copy and paste what **Luís Gustavo Modelli** wrote and use the salary limit to guide the analysis.

This database may reveal:

* Higher salaries
* The contribution of extra remuneration to higher salaries

By the rules of the government no employee could receive more than the state governor salary: R$ 21,631.05


# 1. Data overview 

In [ ]:
# There are some details with the dataset that i needed to handle carefully,i'll mention them.

In [ ]:
# first of all, we need to handle with the variables format.
df.info()

In [ ]:
df = df.convert_objects(convert_numeric=True)

In [ ]:
#Now it looks correct,let's proceed.
df.info()

In [ ]:
df.head()

# 1.1  Data Cleaning

There are some weird values over the variable **total_salary** ,ie., there're negative values for a month of work income, so i think that there is some explanation for this,but  i will work here only with **total_salary** > 0 .

In [ ]:
df = df[df.total_salary>0]

First of all i've notice that there are some null values over the variables : Month_salary,13_salary,eventual_salary	indemnity,extra_salary,discount_salary. but the **total_salary** was **not necessarily null**, as you can see here : 

In [ ]:
df[(df['Month_salary'] ==0) & (df['13_salary']==0) & 
    (df['eventual_salary']==0) & (df['indemnity']==0)&(df['extra_salary']==0) &
    (df['discount_salary']==0)].tail(4)

After looking at the **Job** description for these guys, i did  some research on internet and found that this might be a month pension for dead public employee,which i think it's fair to remove them for a dataset analysis. 
 


In [ ]:
ddf = df[(df['Month_salary'] ==0) & (df['13_salary']==0) & 
    (df['eventual_salary']==0) & (df['indemnity']==0)&(df['extra_salary']==0) &
    (df['discount_salary']==0)].copy()

all_list = df.index
ddf_list =ddf.index
actual_list = all_list.difference(ddf_list)

df = df[df.index.isin(actual_list)].copy()

# Missing Data

In [ ]:
#Checking if our dataset has missing data:
sns.heatmap(df.isnull(),cbar=False)

In [ ]:
missing_data(df)

Looks like we're ok to proceed :)

# 1.2 Data Visualization

Finally we can look some interesting things about the data :), fist let's look the highest salaries by **Sector** and then by **Job**:

In [ ]:
# Limit salary 
limit = 21631.05

In [ ]:
higher_values = df.groupby(['sector'])['total_salary'].mean().nlargest(15).values
higher_index = df.groupby(['sector'])['total_salary'].mean().nlargest(15).index

In [ ]:
#the 15 higher Sector mean salaries 
plt.figure(figsize=(18,14))
plt.ylabel("Sector")
plt.xlabel("Total Salary")
pall = sns.color_palette("hls", 8)
sns.barplot(y=higher_index,x=higher_values,palette=pall)

looks like the **total_salary** mean for **sector** variable follows the law,great.

Now let's look at the highest Job mean salary:

In [ ]:
higher_values = df.groupby(['job'])['total_salary'].mean().nlargest(15).values
higher_index = df.groupby(['job'])['total_salary'].mean().nlargest(15).index

In [ ]:
#the 15 higher Job mean salaries
plt.figure(figsize=(18,14))
plt.ylabel("Job")
plt.xlabel("Total Salary")
pall =sns.color_palette("hls", 8)
sns.barplot(y=higher_index,x=higher_values,palette=pall)
plt.axvline(limit,linewidth=4,color='orange')

The **vertical orange** line represent the **amount of salary limit ** by law.

Interesting result,but the mean for both variables don't show us much about how high from the limit some guys are 
earning,let's look at the 100 highest total salaries,ie.  after all taxes , the 'real' salary.

In [ ]:
index_larg = df.total_salary.nlargest(100).index
job_larg = df[df.index.isin(index_larg)].sort_values(by='total_salary',ascending=False)['job'].values
salar_larg =df[df.index.isin(index_larg)].sort_values(by='total_salary',ascending=False)['total_salary'].values

plt.figure(figsize=(18,14))
plt.ylabel("Job")
plt.xlabel("Total Salary")
sns.barplot(y=job_larg,x=salar_larg,palette='GnBu_d')
plt.axvline(limit,linewidth=4,color='orange')

Now i should look at distributions for a general overview to analyse **total_salary** ,i can do this  by two important graphs:

* **Histogram**: which will allow me to see the simetry (Skewness) and if there are outliers (Kurtosis).
* **QQ Plot** : The QQ plot is a good way to see if our data behaviour follows a normal distribution. 

In [ ]:
#let's look at the distribution of total_salary

sns.distplot(df.total_salary,fit=norm,fit_kws={'color':"red"})
fig = plt.figure()
res = stats.probplot(df.total_salary, plot=plt)

In [ ]:
dist_info(df,'total_salary')

Ok,  looks like we've got a high number of outliers,and the simetry as well is not so good, in that case we should do a transformation:

In [ ]:
df['log_salary'] = df.total_salary.apply(np.log)
# with this transformation,let's look at it again and see if it worked : 

In [ ]:

sns.distplot(df.log_salary,fit=norm,fit_kws={'color':"red"})
fig = plt.figure()
res = stats.probplot(df.log_salary, plot=plt)

In [ ]:
dist_info(df,'log_salary')

Definitely we've got a better result,but still looks like the problem with outliers persist

# Delta Salary variable

Motivated by an *income tax* like effect,  i decided to create a variable that gives  the percentage of the tax from  receveid money, for each one of these guys. the variable it's simply 1 minus  the ratio of **total_salary** ( ts ) by the sum over the income  salaries.

$ \Delta S = 100 (1 - \frac{ts}{\sum income})$

In [ ]:
v = []
print("Start")
print("="*90)
for i in range(len(df)):
    summ = df.iloc[i:i+1,3:9].sum(axis=1).values[0]
    tot = df.iloc[i:i+1,9:10]['total_salary'].values[0]
    v.append((1 - tot/summ)*100)
print("="*90)
print("End")

In [ ]:
df['Delta_S'] = v

i create a function to identify better intervals of **Delta_S** values called **Delta_mapping** , applying it we've got :

In [ ]:
df['Delta Interval'] = df.Delta_S.apply(Delta_mapping)

In [ ]:
df['Delta Interval'].value_counts()

In [ ]:
plt.figure(figsize=(15,9))
sns.barplot(x=df['Delta Interval'].unique(),y=df['Delta Interval'].value_counts(normalize=True)*100)

So let's look how this **Delta_S** behave:


In [ ]:
sns.distplot(df['Delta_S'])

It's really strange that we've got negative values for Delta_S .,these values need a better treatment and i'll ,for now, ignore them and take only look at the interval [0,100], because we've seen that the proportion of Delta Interval over 100 is tiny.

In [ ]:
df = df[df.index!=990847].copy()
sns.distplot(df[(df.Delta_S>=0) & (df.Delta_S <=100)]['Delta_S'],fit=norm,fit_kws={'color':"red"})

In [ ]:
dist_info(df[(df.Delta_S>=0) & (df.Delta_S <=100)],'Delta_S')

To finish this kernel, i will create a dummy variable to identify those who earn a salary higher than the limit value and those who don't.

In [ ]:
df['Over_Limit'] = df.total_salary.apply(lambda x:1 if x > limit else 0)

In [ ]:
sns.distplot(df[df.Over_Limit==1]['log_salary'],color='green',kde_kws={'label':"Over Limit"})
sns.distplot(df[df.Over_Limit==0]['log_salary'],color='blue',kde_kws={'label':"Under Limit"})
plt.legend()
plt.xlim(5,13)
plt.xlabel("Log_Salary",fontsize=12)
plt.ylabel("Limit Density",fontsize=12)

In [ ]:
print("Proportion of ID's with Salary under the limit : {} %".format(100*len(df[df.Over_Limit==0])/len(df)))
print("Proportion of ID's with Salary over the limit : {} %".format(100*len(df[df.Over_Limit==1])/len(df)))

Thank you for your attention, and if you have any critique or suggestion about it , please point it out.

# Thanks 
i would like to thank you** Luiz Leme** for the awesome code to fix the line problem with the dataset, and **Luís Gustavo Modelli** for this amazing dataset ! thank you guys.
